In [30]:
# Import the pandas library for data manipulation and analysis
import pandas as pd

# Import the datetime module to work with date and time objects
from datetime import datetime

# Mount Google Drive to access files stored in your Google Drive account
from google.colab import drive
drive.mount('/content/drive')  # This will prompt you to authorize access and mount the drive at the specified path


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Load the financial information dataset from Google Drive
financial_info = pd.read_csv("/content/drive/MyDrive/Skygeni_Dataset/finanical_information.csv")

# Load the client payment details dataset from Google Drive
payment_info = pd.read_csv("/content/drive/MyDrive/Skygeni_Dataset/payment_information.csv")

# Load the subscription information dataset from Google Drive
subscription_info = pd.read_csv("/content/drive/MyDrive/Skygeni_Dataset/subscription_information.csv")

# Load the industry and client details dataset from Google Drive
industry_info = pd.read_csv("/content/drive/MyDrive/Skygeni_Dataset/industry_client_details.csv")


In [32]:
# Filter clients who belong to the 'Finance Lending' or 'Block Chain' industries,
# then count the number of unique client IDs
finance_blockchain_clients = industry_info[industry_info['industry'].isin(['Finance Lending', 'Block Chain'])]['client_id'].nunique()

# Print the total number of unique clients in the specified industries
print(f"Number of Finance Lending and Blockchain clients: {finance_blockchain_clients}")


Number of Finance Lending and Blockchain clients: 47


In [33]:
# Merge subscription and industry data on 'client_id' to link each subscription with its corresponding industry
merged_data = pd.merge(subscription_info, industry_info, on='client_id', how='left')

# Group the merged data by industry and calculate the average renewal rate for each industry
# (Assumes 'renewed' column is 1 for renewed and 0 for not renewed)
renewal_rates = merged_data.groupby('industry')['renewed'].mean().sort_values(ascending=False)

# Identify the industry with the highest renewal rate
highest_renewal_industry = renewal_rates.idxmax()

# Get the corresponding renewal rate value
highest_renewal_rate = renewal_rates.max()

# Print the result in a nicely formatted message
print(f"Industry with highest renewal rate: {highest_renewal_industry} with rate {highest_renewal_rate:.2%}")

Industry with highest renewal rate: Gaming with rate 72.73%


In [34]:
# Convert 'start_date' in subscription_info to datetime format
subscription_info['start_date'] = pd.to_datetime(subscription_info['start_date'])

# Convert 'start_date' and 'end_date' in financial_info to datetime format
financial_info['start_date'] = pd.to_datetime(financial_info['start_date'])
financial_info['end_date'] = pd.to_datetime(financial_info['end_date'])

# Filter the subscriptions that were renewed
renewed_subs = subscription_info[subscription_info['renewed'] == True]

# Perform an asof merge to align each renewed subscription's start_date
# with the closest (nearest earlier or later) financial data point
merged_fin_sub = pd.merge_asof(
    renewed_subs.sort_values('start_date'),
    financial_info.sort_values('start_date'),
    left_on='start_date',
    right_on='start_date',
    direction='nearest'
)

# Calculate the average inflation rate for the periods during which subscriptions were renewed
avg_inflation_renewed = merged_fin_sub['inflation_rate'].mean()

# Print the result with two decimal places
print(f"Average inflation rate during renewed subscriptions: {avg_inflation_renewed:.2f}%")


Average inflation rate during renewed subscriptions: 4.86%


In [35]:
# Convert 'payment_date' column to datetime format using the specified format (e.g., '3/15/2022')
payment_info['payment_date'] = pd.to_datetime(payment_info['payment_date'], format='%m/%d/%Y')

# Extract the year from the payment date and store it in a new column
payment_info['year'] = payment_info['payment_date'].dt.year

# Group the data by year and calculate the median amount paid for each year
median_amount_by_year = payment_info.groupby('year')['amount_paid'].median()

# Print the median payment amounts for each year
print("Median amount paid each year:")
for year, median in median_amount_by_year.items():
    print(f"Year {year}: ${median:.2f}")


Median amount paid each year:
Year 2018: $235.70
Year 2019: $360.90
Year 2020: $284.50
Year 2021: $306.80
Year 2022: $288.00
